In [8]:
import csv
import numpy as np
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/bigPatentData.csv", header=None)

In [ ]:
df.head()

,0,1,2,3,4
0,US-4074364-A,Human Necessities,"a hold - down device for shirts , blouses , et...","referring to the drawings , the hold - down de...",a
1,US-2017275338-A1,Human Necessities,dengue fever and dengue hemorrhagic fever are ...,"as used herein and except as noted otherwise ,...",a
2,US-10157-P,Human Necessities,a nerbium oleander plant which has a intermedi...,the present invention relates to a new and dis...,a
3,US-7875075-B2,Human Necessities,an implant made of at least two different mate...,the following description is intended to be re...,a
4,US-5048463-A,Human Necessities,a control system for accessories used with sma...,the preferred embodiment for the flush control...,a


In [2]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
!pip install pyspellchecker

In [13]:
import string
from spellchecker import SpellChecker

stop_words = set(stopwords.words('english'))
def remove_stopwords(data):
  new_data = []
  for w in data:
    if w not in stop_words:
      new_data.append(w)
  return new_data

def remove_punc(data):
  new_data = []
  for w in data:
    if w not in string.punctuation:
      new_data.append(w)
  return new_data

def spellcheck(data):
  new_data = []
  spell = SpellChecker()
  for w in data:
    new_word = spell.correction(w)
    new_data.append(new_word)
  return new_data

def lemmatizer(data):
  new_data = []
  lemm = WordNetLemmatizer()
  for w in data:
    new_word = lemm.lemmatize(w)
    new_data.append(new_word)
  return new_data

def preprocess(data):
  data = data.lower()
  data = word_tokenize(data)
  data = remove_stopwords(data)
  data = remove_punc(data)
  data = spellcheck(data)
  data = lemmatizer(data)
  return data

def preprocess_data(raw_text1):
  text = []
  for sent in raw_text1:
    new_sent = preprocess(sent)
    new_sent = " ".join(new_sent) 
    text.append(new_sent)
  return text

In [ ]:
import pickle
raw_text = df[3]

text = preprocess_data(raw_text)

pickle.dump(text, open("/content/drive/MyDrive/preproc_text_PROJECT2","wb"))

In [5]:
import pickle
text = pickle.load(open("/content/drive/MyDrive/preproc_text_PROJECT2","rb"))

In [6]:
!pip install rank_bm25

In [9]:
patents = []
for i in range(len(text)):
  pat = [ df[0][i],df[1][i],df[2][i], df[3][i] ]
  patents.append(pat) 

In [17]:
from rank_bm25 import BM25Okapi
def retrieve_patents(query):
  query = preprocess_data([query])[0]
  tokenized_text = [doc.split(" ") for doc in text]
  tokenized_query = query.split(" ")
  bm25 = BM25Okapi(tokenized_text)
  doc_scores = bm25.get_scores(tokenized_query)
  retrieved_docs = bm25.get_top_n(tokenized_query, patents, n=10)
  return retrieved_docs

In [20]:
query = "clth drying machine"
lst = retrieve_patents(query)
for i in lst:
  print(i)

cloth drying machine
['US-2002183743-A1', 'Human Necessities', 'a urological resectoscope including a main body carrying a tubular shaft , an optical guide plate arranged at a fixed distance proximally of the main body , a sliding body longitudinally movably guided between the main body and the optical guide plate and a spring bridge comprising two levers connected pivotally together at one end and acted on by a spring , the other ends of which are pivotally connected to the sliding body on the one hand and to the optical guide plate or the main body on the other hand . one of the levers is pivotally connected to only one side of the sliding body .', 'referring firstly to fig1 to 3 , the resectoscope includes a main body 1 secured to an optical system guide tube 2 , secured to which at a proximal spacing from the main body 1 there is also an optical guide plate 3 , the purpose of which is well known , namely to guide the optical system and / or to facilitate the introduction of the opt

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
Vect = TfidfVectorizer(ngram_range=(1,1), max_features=10000)
Vect.fit(text)

query_vec = Vect.transform([query])
query_vec = query_vec.toarray()

docs = []
for p in lst:
  docs.append(p[3])

docs_vec = Vect.transform(docs)
docs_vec = docs_vec.toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

coss = []
for vec in docs_vec:
  print(cosine_similarity(query_vec, [vec]))
  coss.append(cosine_similarity(query_vec, [vec]))

[[0.08902785]]
[[0.51537861]]
[[0.2600056]]
[[0.03525153]]
[[0.29870386]]
[[0.12684182]]
[[0.29017945]]
[[0.17505674]]
[[0.23089027]]
[[0.29061829]]


In [ ]:
print("Average cosine score for query:",sum(coss)/10)

Average cosine score for query: [[0.2311954]]
